In [6]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [7]:
df = pd.read_csv("../../data/train.csv")

In [8]:
df.drop(columns=['id'], inplace=True)
df["outcome"] = df["outcome"].replace({"died": 0, "lived": 1, "euthanized": 2})
df.fillna('empty', inplace=True)
df = pd.get_dummies(df)

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X = df.drop(columns=["outcome"])
y = df["outcome"]

In [13]:
X.shape

(1235, 83)

In [25]:
# First define baseline model. Then use it in Keras Classifier for the training
def baseline_model():
    # Create model here
    model = Sequential()
    model.add(Dense(83, input_dim = 83, activation = 'relu')) # Rectified Linear Unit Activation Function
    model.add(Dense(15, activation = 'relu'))
    model.add(Dense(3, activation = 'softmax')) # Softmax for multi-class classification
    # Compile model here
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [27]:
model = baseline_model()

In [31]:
X.values

array([[530001, 38.1, 132.0, ..., True, True, False],
       [533836, 37.5, 88.0, ..., True, True, False],
       [529812, 38.3, 120.0, ..., True, True, False],
       ...,
       [529685, 37.5, 84.0, ..., True, False, True],
       [534784, 38.1, 70.0, ..., True, True, False],
       [528548, 38.1, 54.0, ..., True, False, True]], dtype=object)

In [33]:
model.fit(X.values, y.values)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [29]:
estimator = KerasClassifier(model=baseline_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

ValueError: 
All the 10 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "c:\GitHub\Health-Outcomes-of-Horses\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\GitHub\Health-Outcomes-of-Horses\venv\lib\site-packages\scikeras\wrappers.py", line 1491, in fit
    super().fit(X=X, y=y, sample_weight=sample_weight, **kwargs)
  File "c:\GitHub\Health-Outcomes-of-Horses\venv\lib\site-packages\scikeras\wrappers.py", line 760, in fit
    self._fit(
  File "c:\GitHub\Health-Outcomes-of-Horses\venv\lib\site-packages\scikeras\wrappers.py", line 928, in _fit
    self._fit_keras_model(
  File "c:\GitHub\Health-Outcom